In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 145.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 137.9 MB/

In [2]:
# ingest.py
import os
import json
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
def load_json_knowledge_base(filepath="knowledge_base.json"):
    """
    Loads pre-chunked documents with metadata from a single JSON file.
    The JSON file should be a list of objects, where each object has:
    - "chunk_id": A unique string identifier.
    - "chunk_text": The text content of the chunk.
    - "metadata": A dictionary of metadata.
    """
    print(f"Loading knowledge from JSON file: '{filepath}'...")
    if not os.path.exists(filepath):
        print(f"Error: The file '{filepath}' was not found. Please create it.")
        return []

    try:
        # Ensure the file is read with UTF-8 encoding for Farsi characters
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"Successfully loaded {len(data)} knowledge chunks.")
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {filepath}. Please check the file format. Details: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred while reading {filepath}: {e}")
        return []

In [4]:
def run():
    """
    Main function to ingest documents from JSON and store them in ChromaDB.
    """
    print("--- Starting Knowledge Base Ingestion from JSON ---")

    # 1. Load documents from JSON. This data is already chunked.
    documents_data = load_json_knowledge_base()
    if not documents_data:
        print("No documents to ingest. Exiting.")
        return

    # Extract the necessary components for ChromaDB
    original_texts = [doc["chunk_text"] for doc in documents_data]
    document_ids = [doc["chunk_id"] for doc in documents_data]
    metadatas = [doc["metadata"] for doc in documents_data]

    # ENRICH FOR EMBEDDING
    enriched_texts = []
    for doc in documents_data:
        meta = doc["metadata"]
        # Build natural Farsi context
        context = f"[بخش: {meta.get('source_title')}]"
        if meta.get("parent_category"):
            context += f" [زیرمجموعه: {meta['parent_category']}]"
        enriched = f"{context}\n{doc['chunk_text']}"
        enriched_texts.append(enriched)



    # --- VERIFICATION STEP 1: Inspect the loaded data ---
    print("\n--- Verification: Inspecting first loaded chunk ---")
    if documents_data:
        print(f"Chunk ID: {document_ids[0]}")
        print(f"Chunk Text: \"{original_texts[0][:150]}...\"")
        print(f"Chunk Metadata: {metadatas[0]}")
    print("------------------------------------------------\n")

    # 2. Initialize the embedding model
    print("Initializing embedding model (this may take a moment)...")
    model_name = "paraphrase-multilingual-mpnet-base-v2"
    embedding_model = SentenceTransformer(model_name)
    print("Embedding model initialized.")

    # 3. Setup ChromaDB client
    client = chromadb.PersistentClient(path="./chroma_db")

    # 4. Create or get the collection
    collection_name = "farsi_rag_collection"
    print(f"Setting up ChromaDB collection: '{collection_name}'...")
    # Delete the collection if it already exists to ensure a fresh start
    if collection_name in [c.name for c in client.list_collections()]:
        client.delete_collection(name=collection_name)
        print(f"Deleted existing collection '{collection_name}' for a fresh start.")
    collection = client.get_or_create_collection(name=collection_name)
    print("Collection setup complete.")

    # 5. Generate embeddings for all document chunks
    print("Generating embeddings for all document chunks...")
    embeddings = embedding_model.encode(enriched_texts, show_progress_bar=True)

    # --- VERIFICATION STEP 2: Inspect the embeddings ---
    print("\n--- Verification: Inspecting embeddings ---")
    print(f"Shape of embeddings array: {embeddings.shape}")
    print(f"Sample of first embedding vector: {embeddings[0][:5]}...")
    print("------------------------------------------\n")

    # 6. Add to ChromaDB collection (NOW WITH METADATA!)
    print("Adding documents with their metadata to ChromaDB...")
    collection.add(
        embeddings=embeddings.tolist(),
        documents=original_texts,
        metadatas=metadatas,  # <-- Here we add the rich metadata
        ids=document_ids
    )

    # --- VERIFICATION STEP 3: Test a query ---
    print("\n--- Verification: Performing a test query ---")
    test_query = "خدمات شرکت چیست؟"
    print(f"Querying with text: \"{test_query}\"")

    test_query_embedding = embedding_model.encode(test_query).tolist()

    results = collection.query(
        query_embeddings=[test_query_embedding],
        n_results=1
    )

    retrieved_doc = results['documents'][0][0]
    retrieved_meta = results['metadatas'][0][0]
    print(f"Retrieved document: \"{retrieved_doc[:150]}...\"")
    print(f"Retrieved metadata: {retrieved_meta}")
    print("--------------------------------------------\n")

    print("\n--- Ingestion Complete ---")
    print(f"Successfully added {len(documents_data)} documents to the '{collection_name}' collection.")
    print("You can now run 'app.py' to start the chatbot.")

In [5]:
run()

--- Starting Knowledge Base Ingestion from JSON ---
Loading knowledge from JSON file: 'knowledge_base.json'...
Successfully loaded 99 knowledge chunks.

--- Verification: Inspecting first loaded chunk ---
Chunk ID: contact_us_001
Chunk Text: "راه‌های تماس با ما به شرح زیر است:
- تلفن همراه: ۰۹۹-۸۱۸۱-۸۹۷۸
- تلفن ثابت: ۰۱۱-۴۲۴۳-۳۴۴۳
- فکس: ۰۱۱-۴۲۴۳-۳۴۴۴
- ایمیل: info@tabenergy.ir
- آدرس: مازن..."
Chunk Metadata: {'source_title': 'تماس با ما', 'source_url': 'tabenergy.ir/contact/'}
------------------------------------------------

Initializing embedding model (this may take a moment)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model initialized.
Setting up ChromaDB collection: 'farsi_rag_collection'...
Collection setup complete.
Generating embeddings for all document chunks...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]


--- Verification: Inspecting embeddings ---
Shape of embeddings array: (99, 768)
Sample of first embedding vector: [-0.03597984 -0.03558147 -0.01376151 -0.04435125  0.12509175]...
------------------------------------------

Adding documents with their metadata to ChromaDB...

--- Verification: Performing a test query ---
Querying with text: "خدمات شرکت چیست؟"
Retrieved document: "سوال: آدرس شرکت کجاست؟
پاسخ: آدرس ما: مازندران، سوادکوه شمالی، شهرک صنعتی بشل، فاز ۲، همت ۷...."
Retrieved metadata: {'source_title': 'آدرس شرکت کجاست؟', 'parent_category': 'پرسش و پاسخ های رایج', 'source_url': 'tabenergy.ir/contact/'}
--------------------------------------------


--- Ingestion Complete ---
Successfully added 99 documents to the 'farsi_rag_collection' collection.
You can now run 'app.py' to start the chatbot.
